In [1]:
import requests
import numpy as np

def get_login_url(player_id):
    return f'http://95.217.177.249/casino/createacc?id={player_id}'

def get_bet_url(mode, player_id, cash, number):
    return f'http://95.217.177.249/casino/play{mode}?id={player_id}&bet={cash}&number={number}'

In [2]:
player_id = 1007
resp = requests.get(get_login_url(player_id)).json()

In [3]:
resp

{'id': '1007', 'money': 1000, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}

## 1. Linear congruential generator

In [4]:
mode = 'Lcg'

In [5]:
def get_next_value(mode, player_id):
    resp = requests.get(get_bet_url(mode, player_id, 1, 1)).json()
    print(resp)
    return np.uint32(resp['realNumber'])

def get_prediction(last, a, c):
    result = (a * last + c) % 2**32
    return np.int32(result)
        

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(b, n):
    g, x, _ = egcd(b, n)
    return x % n if g == 1 else None

In [6]:
k1, k2, k3 = [get_next_value(mode, player_id) for _ in range(3)]
print(k1, k2, k3)

{'message': 'You lost this time', 'account': {'id': '1007', 'money': 999, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': 224727455}
{'message': 'You lost this time', 'account': {'id': '1007', 'money': 998, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': -400739726}
{'message': 'You lost this time', 'account': {'id': '1007', 'money': 997, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': -1792676055}
224727455 3894227570 2502291241


In [7]:
mod = modinv(k1 - k2, 2**32)

while mod is None:
    k1, k2, k3 = k2, k3, get_next_value(mode, player_id)
    mod = modinv(k1 - k2, 2**32)

a = (k2 - k3) * mod % 2**32
c = (k2 - a * k1) % 2**32
print(a, c)

1664525 1013904223


/home/vasniktel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in uint_scalars
  """Entry point for launching an IPython kernel.


In [9]:
last = k3
for i in range(5):
    last = get_prediction(last, a, c)
    print(requests.get(get_bet_url(mode, player_id, 100, last)).json())

{'message': 'Yay! https://docs.google.com/document/d/1E_ltXUqvmmWeb3Dl3Qsyexsy5V7M6Lb1kvkXiXz9sks/edit?usp=sharing https://docs.google.com/document/d/1qsNXIqxQEs4Xbz5ye6z0ttrct1qn7zos4Vwl7PzcEK0/edit?usp=sharing', 'account': {'id': '1007', 'money': 100897, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': 1202155124}
{'message': 'Yay! https://docs.google.com/document/d/1E_ltXUqvmmWeb3Dl3Qsyexsy5V7M6Lb1kvkXiXz9sks/edit?usp=sharing https://docs.google.com/document/d/1qsNXIqxQEs4Xbz5ye6z0ttrct1qn7zos4Vwl7PzcEK0/edit?usp=sharing', 'account': {'id': '1007', 'money': 200797, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': 1598408515}
{'message': 'Yay! https://docs.google.com/document/d/1E_ltXUqvmmWeb3Dl3Qsyexsy5V7M6Lb1kvkXiXz9sks/edit?usp=sharing https://docs.google.com/document/d/1qsNXIqxQEs4Xbz5ye6z0ttrct1qn7zos4Vwl7PzcEK0/edit?usp=sharing', 'account': {'id': '1007', 'money': 300697, 'deletionTime': '2020-12-05T09:39:21.2971753Z'}, 'realNumber': 1441383366}
{'me

## 2. Mersenne Twister 19937

In [10]:
mode = 'Mt'

player_id = 2016
player = requests.get(get_login_url(player_id)).json()

In [11]:
player

{'id': '2016', 'money': 1000, 'deletionTime': '2020-12-05T09:39:36.1405238Z'}

In [12]:
first_value = get_next_value(mode, player_id)
first_value

{'message': 'You lost this time', 'account': {'id': '2016', 'money': 999, 'deletionTime': '2020-12-05T09:39:36.1405238Z'}, 'realNumber': 1128874927}


1128874927

In [13]:
from datetime import datetime, timezone

def infer_mt_seed(delta, first_value):
    curr_time = int(datetime.now(timezone.utc).timestamp())
    for i in range(*delta):
        mt = np.random.RandomState(curr_time + i)
        if mt.randint(0, 2**32) == first_value:
            return curr_time + i
    
    assert False, 'Unable to find the seed'

In [14]:
seed = infer_mt_seed(delta=(-600, 600), first_value=first_value)
seed

1607157578

In [16]:
rng = np.random.RandomState(seed)
rng.randint(0, 2**32)
for i in range(10):
    print(requests.get(get_bet_url(mode, player_id, 100, rng.randint(0, 2**32))).json())

{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/19vgZtvDN4_StEgVEM9MjfxnqfayByLNMD7PFJgvZv7c/edit?usp=sharing", 'account': {'id': '2016', 'money': 100899, 'deletionTime': '2020-12-05T09:39:36.1405238Z'}, 'realNumber': 2931529878}
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/19vgZtvDN4_StEgVEM9MjfxnqfayByLNMD7PFJgvZv7c/edit?usp=sharing", 'account': {'id': '2016', 'money': 200799, 'deletionTime': '2020-12-05T09:39:36.1405238Z'}, 'realNumber': 2175567922}
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/19vgZtvDN4_StEgVEM9MjfxnqfayByLNMD7PFJgvZv7c/edit?usp=sharing", 'account': {'id': '2016', 'money': 300699, 'deletionTime': '2020-12-05T09:39:36.1405238Z'}, 'realNumber': 3436509148}
{'message': "Yay! It's different from the first one: https://docs.google.com/document/d/19vgZtvDN4_StEgVEM9MjfxnqfayByLNMD7PFJgvZv7c/edit?usp=sharing", 'account': {'id': '2016', 'money': 400

## 3. MT19937 with strong seed

In [17]:
mode = 'BetterMt'

player_id = 3001
player = requests.get(get_login_url(player_id)).json()

In [18]:
player

{'id': '3001', 'money': 1000, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}

In [19]:
def untemper(value):
    (w, n, m, r) = (32, 624, 397, 31)
    a = 0x9908B0DF
    (u, d) = (11, 0xFFFFFFFF)
    (s, b) = (7, 0x9D2C5680)
    (t, c) = (15, 0xEFC60000)
    l = 18
    f = 1812433253
    
    value = np.uint32((value >> l) ^ value)
    value = np.uint32(((value << t) & c) ^ value)
    value = np.uint32(((value << s) & 0x00001680) ^ value)
    value = np.uint32(((value << s) & 0x000c4000) ^ value)
    value = np.uint32(((value << s) & 0x0d200000) ^ value)
    value = np.uint32(((value << s) & 0x90000000) ^ value)
    value = np.uint32(((value >> u) & 0xffc00000) ^ value)
    value = np.uint32(((value >> u) & 0x003ff800) ^ value)
    value = np.uint32(((value >> u) & 0x000007ff) ^ value)
    return value

In [20]:
arr = np.array(
    [untemper(get_next_value(mode, player_id)) for _ in range(624)])
mt = np.random.RandomState()
mt.set_state(('MT19937', arr, 624))

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 999, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3856786982}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 998, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3529934043}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 997, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3002007104}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 996, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 4275516169}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 995, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1197996655}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 994, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3534260421}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 993, 'deletionTime': '2020-12-05T09:4

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 943, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1339520089}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 942, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 257615742}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 941, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3073296220}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 940, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2889274116}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 939, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3123759143}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 938, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 859793519}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 937, 'deletionTime': '2020-12-05T09:40:

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 887, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 844877874}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 886, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 124336858}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 885, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1128803637}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 884, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2016961695}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 883, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2983432408}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 882, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1845230757}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 881, 'deletionTime': '2020-12-05T09:40:

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 831, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1887862159}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 830, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3574877370}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 829, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 278748653}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 828, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 113447767}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 827, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1851865317}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 826, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3658207938}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 825, 'deletionTime': '2020-12-05T09:40:

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 775, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 4061348119}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 774, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 585084901}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 773, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3929685875}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 772, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1734876240}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 771, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2782162434}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 770, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2173272456}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 769, 'deletionTime': '2020-12-05T09:40

...

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 606, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1483375690}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 605, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3750219563}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 604, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3583722942}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 603, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 76307829}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 602, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2898362111}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 601, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 585702186}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 600, 'deletionTime': '2020-12-05T09:40:0

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 550, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3235903925}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 549, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3224592954}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 548, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1510343697}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 547, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1068606845}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 546, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 438900013}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 545, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3222373737}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 544, 'deletionTime': '2020-12-05T09:40

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 493, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 271824165}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 492, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3071652009}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 491, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 4205915682}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 490, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 597635654}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 489, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3179649770}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 488, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3102586501}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 487, 'deletionTime': '2020-12-05T09:40:

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 437, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3485728737}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 436, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 979788847}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 435, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2181072225}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 434, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1372121360}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 433, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3593519165}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 432, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3565861786}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 431, 'deletionTime': '2020-12-05T09:40

{'message': 'You lost this time', 'account': {'id': '3001', 'money': 381, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1298024627}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 380, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1286571713}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 379, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 423552216}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 378, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 833007139}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 377, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3583722942}
{'message': 'You lost this time', 'account': {'id': '3001', 'money': 376, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 1510207737}


In [21]:
for i in range(15):
    print(requests.get(get_bet_url(mode, player_id, 100, mt.randint(0, 2**32))).json())

{'message': 'Yay!', 'account': {'id': '3001', 'money': 100276, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 746963718}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 200176, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3721757713}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 300076, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2788507875}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 399976, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 3583983506}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 499876, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 2834393091}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 599776, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 735860235}
{'message': 'Yay!', 'account': {'id': '3001', 'money': 699676, 'deletionTime': '2020-12-05T09:40:03.0935161Z'}, 'realNumber': 4096483247}
{'message': 'Yay!', 'account': {'id'